In [ ]:
import numpy as np
import pandas as pd

#Reading the test and the train data

train = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
print('train shape : ', train.shape)
print('test shape : ', test.shape)



In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train = train.set_index('row_id')
test = test.set_index('row_id')

In [ ]:
#Initial EDA 

train.head(10)

In [ ]:
train.info()

In [ ]:
#Check for null values

train.isnull().sum()

In [ ]:
test.isnull().sum()

The dataset does not have any null values

In [ ]:
#Looking at basic stats

#For numeric cols

train.describe().T  #The transpose is used for better view.

In [ ]:
test.describe().T

In [ ]:
#For categorical columns

train.describe(include=['O'])

Thus we can see in the training dataset we have 3 unique countries, 2 unique stores (KaggleMart or KaggleRama) and 3 unique products

In [ ]:
test.describe(include=['O'])

In [ ]:
#Lets look at the frequency distribution of the key categorical variables country, store, product

print(train['country'].value_counts())
print('\n')
print(train['store']. value_counts())
print('\n')
print(train['product'].value_counts())



One interesting aspect is that the product and the country distributions are the same (8766), does that mean a particular country produces just one of those products? Lets check it out.

In [ ]:
train.groupby(['country','product']).size()

In [ ]:
#Looking at the dates feature to understand what period the data belongs to. 

def min_max_dt(df, name='train'):
    #df['date'] = pd.to_datetime(df['date'])
    min_date = df['date'].min()
    max_date = df['date'].max()
    print(f'For the {name} data : Min_date - {min_date} / Max_date - {max_date}')
    return None
          
          
min_max_dt(train, 'train')
min_max_dt(test, 'test')


The train data is from Jan 2015 to Dec 2018, while the test data is from Jan 2019 to Dec 2019

The metric that will be used to evaluate the competition:

In [ ]:

def smape_loss(y_true, y_pred):
    """SMAPE Loss"""
    return np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200

#print(smape_loss(tf.constant([1, 2]), tf.constant([3, 4]))) # should print [100, 66.6667]

In [ ]:
#Credit to https://www.kaggle.com/jaredfeng/tps-jan22-inprog-v5

holiday_path = '../input/holidays-finland-norway-sweden-20152019/Holidays_Finland_Norway_Sweden_2015-2019.csv'

def GetHoliday(holiday_path, df):
    """
    Get a boolean feature of whether the current row is a holiday sale
    """
    
    holiday = pd.read_csv(holiday_path)
    fin_holiday = holiday.loc[holiday.Country == 'Finland']
    swe_holiday = holiday.loc[holiday.Country == 'Sweden']
    nor_holiday = holiday.loc[holiday.Country == 'Norway']
    df['fin holiday'] = df.date.isin(fin_holiday.Date).astype(float)
    df['swe holiday'] = df.date.isin(swe_holiday.Date).astype(float)
    df['nor holiday'] = df.date.isin(nor_holiday.Date).astype(float)
    
    df['holiday'] = np.zeros(df.shape[0]).astype(int)
    df.loc[df.country == 'Finland', 'holiday'] = df.loc[df.country == 'Finland', 'fin holiday']
    df.loc[df.country == 'Sweden', 'holiday'] = df.loc[df.country == 'Sweden', 'swe holiday']
    df.loc[df.country == 'Norway', 'holiday'] = df.loc[df.country == 'Norway', 'nor holiday']
    df.drop(['fin holiday', 'swe holiday', 'nor holiday'], axis=1, inplace=True)
    return df

train = GetHoliday(holiday_path, train)
test = GetHoliday(holiday_path, test)


In [ ]:
# Credit to https://www.kaggle.com/ranjeetshrivastav/tps-jan-21-base-xgb
# and https://www.kaggle.com/bernhardklinger/tps-jan-2022/notebook

def feature_eng(df):
    df['date'] = pd.to_datetime(df['date'])
    df['week']= df['date'].dt.week
    #df['year'] = 'Y' + df['date'].dt.year.astype(str)
    df['quarter'] = 'Q' + df['date'].dt.quarter.astype(str)
    df['day'] = df['date'].dt.day
    df['dayofyear'] = df['date'].dt.dayofyear
    df.loc[(df.date.dt.is_leap_year) & (df.dayofyear >= 60),'dayofyear'] -= 1
    df['weekend'] = df['date'].dt.weekday >=5
    df['weekday'] = 'WD' + df['date'].dt.weekday.astype(str)
    df.drop(columns=['date'],inplace=True)  

feature_eng(train)
feature_eng(test)

In [ ]:
train.columns

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.info()

In [ ]:
train['day'].value_counts()

Steps to take

1. Convert the categorical variables into numeric
2. Align the train and test columns
3. Run the non linear models
4. Stack the non-linear model
5. Predict and submit predictions

In [ ]:
X_train = train.drop(['num_sold'],axis=1)
y_train = train['num_sold']
X_test = test   


In [ ]:
X_train.columns

Step 1 - Converting categorical variables into numeric & Step 2 - Align the train and test columns


In [ ]:
X_train_ohe = pd.get_dummies(X_train)
X_test_ohe = pd.get_dummies(X_test)
final_train, final_test = X_train_ohe.align(X_test_ohe,join='left', axis=1)

#Please refer to this excellent notebook from Dans Becker to understand how to align categorical columns



In [ ]:
final_train.head()

In [ ]:
final_test.head()

3. Run the non linear models (also test scores with linear models)

In [ ]:
#Standardising the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(final_train)
X_test_scaled = scaler.transform(final_test)



In [ ]:
#Creating a custom scoring for cross_validation

from sklearn.metrics import fbeta_score, make_scorer

# Credit to https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

smape_score = make_scorer(SMAPE, greater_is_better=False)


Since we want to use log for the dependent variable, we can use scikitlearn's TransformedRegressor.

https://scikit-learn.org/stable/modules/generated/sklearn.compose.TransformedTargetRegressor.html



In [ ]:
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import cross_val_score


tt_xgb = TransformedTargetRegressor(regressor=XGBRegressor(),
                                   func=np.log, inverse_func=np.exp)

scores_xgb = cross_val_score(tt_xgb, X_train_scaled, y_train, scoring=smape_score, cv=5)


tt_lgb = TransformedTargetRegressor(regressor=LGBMRegressor(),
                                   func=np.log, inverse_func=np.exp)

scores_lgb = cross_val_score(tt_lgb, X_train_scaled, y_train, scoring=smape_score, cv=5)

tt_gbr = TransformedTargetRegressor(GradientBoostingRegressor(),
                                   func=np.log, inverse_func=np.exp)

scores_gbr = cross_val_score(tt_gbr, X_train_scaled, y_train, scoring=smape_score, cv=5)


tt_ridge = TransformedTargetRegressor(regressor=Ridge(),
                                   func=np.log, inverse_func=np.exp)

scores_ridge = cross_val_score(tt_ridge, X_train_scaled, y_train, scoring = smape_score, cv = 5)


tt_lasso = TransformedTargetRegressor(regressor=Lasso(),
                                   func=np.log, inverse_func=np.exp)

scores_lasso = cross_val_score(tt_lasso, X_train_scaled, y_train, scoring = smape_score, cv = 5)


tt_elasticnet = TransformedTargetRegressor(regressor=ElasticNet(),
                                   func=np.log, inverse_func=np.exp)

scores_elasticnet = cross_val_score(tt_elasticnet, X_train_scaled, y_train, scoring = smape_score, cv = 5)

tt_svr = TransformedTargetRegressor(regressor = SVR(C=20, epsilon = 0.008, gamma = 0.0003))

scores_svr = cross_val_score(tt_svr, X_train_scaled, y_train, scoring = smape_score, cv = 5)



In [ ]:
print(scores_xgb)
print(scores_lgb)
print(scores_gbr)
print(scores_ridge)
print(scores_lasso)
print(scores_elasticnet)
print(scores_svr)

Clearly the linear models dont do well. Lets stack the non linear models.

In [ ]:

stack_gen = StackingCVRegressor(regressors=(tt_xgb, tt_lgb, tt_gbr),
                                meta_regressor=tt_xgb,
                                use_features_in_secondary=True)
scores_stack = cross_val_score(stack_gen, X_train_scaled, y_train, scoring = smape_score, cv = 5)


In [ ]:
print(scores_stack)

In [ ]:
#Lets fit all the non linear models on the full training data

tt_xgb.fit(X_train_scaled, y_train)
tt_lgb.fit(X_train_scaled, y_train)
tt_gbr.fit(X_train_scaled, y_train)

In [ ]:
#Lets fit the stacked model on the training data
stack_gen.fit(X_train_scaled, y_train)



4. Predict a blended model and submit predictions

In [ ]:
def blend_models_predict(X_test, m1, m2, m3, m4):
    return ((m1 * tt_xgb.predict(X_test)) + \
            (m2 * tt_lgb.predict(X_test)) + \
            (m3 * tt_gbr.predict(X_test)) + \
            (m4 * stack_gen.predict(X_test))
            )

In [ ]:

test_y_pred = blend_models_predict(X_test_scaled, 0.1, 0.3, 0.3, 0.3)



In [ ]:
test_y_pred

In [ ]:
#submission

assert(len(test.index)==len(test_y_pred))

submission_df = pd.DataFrame(list(zip(test.index, test_y_pred)), columns=['row_id', 'num_sold'])

submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df